In [ ]:
from __future__ import annotations

from pathlib import Path
import json
import random

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as T
from PIL import Image

In [ ]:
seed = 42
random.seed(seed)
torch.manual_seed(seed)

device = "cpu"
print(f"Using {device} device")

In [ ]:
class DogsDataset(Dataset):
    def __init__(self, jsonl_path: str | Path, transform=None, class_to_idx: dict[str, int] | None = None):
        self.jsonl_path = Path(jsonl_path)
        self.transform = transform
        with self.jsonl_path.open("r", encoding="utf-8") as f:
            self.items = [json.loads(line) for line in f if line.strip()]

        if class_to_idx is None:
            classes = sorted({item["class_name"] for item in self.items})
            self.class_to_idx = {c: i for i, c in enumerate(classes)}
        else:
            self.class_to_idx = dict(class_to_idx)

        self.idx_to_class = {v: k for k, v in self.class_to_idx.items()}

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        item = self.items[idx]
        image = Image.open(item["image"]).convert("RGB")
        label = self.class_to_idx[item["class_name"]]
        if self.transform is not None:
            image = self.transform(image)
        return image, label

In [ ]:
image_size = 160
batch_size = 16

train_transform = T.Compose([
    T.RandomResizedCrop(image_size),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])

eval_transform = T.Compose([
    T.Resize(int(image_size * 1.14)),
    T.CenterCrop(image_size),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])

training_data = DogsDataset("data/splits/dogs_train.jsonl", transform=train_transform)
val_data = DogsDataset("data/splits/dogs_val.jsonl", transform=eval_transform, class_to_idx=training_data.class_to_idx)
test_data = DogsDataset("data/splits/dogs_test.jsonl", transform=eval_transform, class_to_idx=training_data.class_to_idx)

train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True, num_workers=0)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=0)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=0)

x, y = next(iter(train_dataloader))
print("train/val/test sizes:", len(training_data), len(val_data), len(test_data))
print("x:", x.shape, "y:", y.shape, "num_classes:", len(training_data.class_to_idx))

In [ ]:
num_classes = len(training_data.class_to_idx)

try:
    weights = torchvision.models.ResNet18_Weights.DEFAULT
    model = torchvision.models.resnet18(weights=weights)
    print("Loaded pretrained ResNet18 weights")
except Exception as e:
    model = torchvision.models.resnet18(weights=None)
    print("Could not load pretrained weights; using random init. Error:", repr(e))

model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

print("Model ready. Output classes:", num_classes)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-2)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        loss_value, current = loss.item(), (batch + 1) * len(X)
        print(f"loss: {loss_value:>7f}  [{current:>5d}/{size:>5d}]")

@torch.no_grad()
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0.0, 0.0
    for X, y in dataloader:
        X, y = X.to(device), y.to(device)
        pred = model(X)
        test_loss += loss_fn(pred, y).item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= max(1, num_batches)
    correct /= max(1, size)
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return 100 * correct

In [ ]:
epoch = 0
epochs = 10

while True:
    print(f"Epoch {epoch+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    val_acc = test(val_dataloader, model, loss_fn)
    if (epochs != 0 and epoch + 1 >= epochs):
        break
    epoch += 1

print("Done!")

In [ ]:
test_acc = test(test_dataloader, model, loss_fn)
print(f"Test acc {test_acc:.2f}%")

In [ ]:
torch.save({"state_dict": model.state_dict(), "class_to_idx": training_data.class_to_idx}, "resnet18_dogs_cpu.pth")
print("Saved to resnet18_dogs_cpu.pth")

In [ ]:
import io, sys, subprocess
from IPython.display import display
from PIL import Image

try:
    import ipywidgets as widgets
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ipywidgets"])
    import ipywidgets as widgets

idx_to_class = {v: k for k, v in training_data.class_to_idx.items()}
model.eval()

uploader = widgets.FileUpload(accept="image/*", multiple=False)
out = widgets.Output()

def _predict_from_bytes(image_bytes: bytes):
    img = Image.open(io.BytesIO(image_bytes)).convert("RGB")
    x = eval_transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        logits = model(x)
        probs = torch.softmax(logits, dim=1)[0]
        values, indices = torch.topk(probs, k=min(5, probs.numel()))
    return [(idx_to_class[int(i)], float(v)) for v, i in zip(values, indices)]

def _get_uploaded_bytes(value):
    if value is None:
        return None
    if isinstance(value, dict):
        if not value:
            return None
        item = next(iter(value.values()))
        if isinstance(item, dict):
            return item.get("content") or item.get("data")
        return getattr(item, "content", None)
    if isinstance(value, (tuple, list)):
        if len(value) == 0:
            return None
        item = value[0]
        if isinstance(item, dict):
            return item.get("content") or item.get("data")
        return getattr(item, "content", None)
    return getattr(value, "content", None)

def _on_upload(change):
    out.clear_output()
    image_bytes = _get_uploaded_bytes(change.get("new"))
    if image_bytes is None:
        return
    preds = _predict_from_bytes(image_bytes)
    with out:
        print("Top predictions:")
        for cls, p in preds:
            print(f"{cls}: {p*100:.2f}%")

uploader.observe(_on_upload, names="value")
display(uploader, out)

In [ ]:
checkpoint = torch.load("resnet18_dogs_cpu.pth")
model.load_state_dict(checkpoint["state_dict"])
print("Model loaded successfully")